`일단 하나씩 봐보도록 하자!`

##### 해당논문('빅데이터 분석을 통한 서울시 골목상권 분석')을 보면, 중요한것들은 다음과 같습니다.    
<img width="677" alt="download" src="https://user-images.githubusercontent.com/91362178/230754357-3658712b-605f-4665-a6b9-67cbad9ef90e.png" width="30" height="500">  
     
---  
일단 아래 상권별 평가지표와 제가 써둔 잡다한 생각들은 한번 읽어만 봐주세용

<img src="https://user-images.githubusercontent.com/91362178/230754614-7ab83ca6-5cce-4bac-be24-50a1016a1a14.JPG" width="600" height="800"/>

![IMG_0487](https://user-images.githubusercontent.com/91362178/230754599-17c0b51d-4f5a-49a2-9c2e-5e53f3e5684e.PNG)


### 분석에 앞서, 새롭게 등장한 인구 개념인 생활인구에 대해서 간략하게 적어두겠습니다.  

* 2023년 1월부터 시행되는 「인구감소지역 지원 특별법」은 생활인구라는 개념을 새로이 도입하였다.
생활인구는 주민등록인구 및 외국인등록인구 외에 지역에 체류하는 인구까지 포함하는 개념이다. 생활
인구는 인구의 이동성까지 반영한 것으로, 향후 실제 행정수요를 반영한 공공서비스 공급 등의 효과가
있을 것으로 기대된다. 다만, 현행 주민등록제도 하에서 이러한 새로운 인구개념의 도입은 정책 현장
에서 혼란을 야기할 우려가 있으므로 정부와 지방자치단체는 사전에 철저한 준비와 시행착오에 대한
대비가 필요할 것이다.   

<img width="696" alt="생활인구_뜻" src="https://user-images.githubusercontent.com/91362178/230756996-1c0e1249-022d-45d6-9862-0518c9630a4f.png" width="600" height="600">

### 위의 자료들과 제가 생각했을때 필요한 column들, 구하고자 하는 칼럼들은 다음과 같습니다.  
———  

1. (상권-생활인구 에서 따올 애들)   
기준_년월_코드  (모든 데이터를 뒤져본 결과 2017~있는 애들 다수, 2018부터 있는애, 2014부터 있는애 소수 이렇게 있었음. 깔끔하게 2019년,18년부터 보는게 괜찮을듯)  
상권_구분_코드_명  
상권_코드  
상권_코드_명  
생활인구 수   

——  

2. (상권-집객시설)  
집객시설_수  

——  

3. (상권-점포)  
서비스_업종_코드  
서비스_업종_코드_명   
 
——  

4. (상권-아파트)  
아파트_평균_시가  
아파트_평균_면적   

——  

5. (상권 - 추정매출에서 분기별로 따와서 더하자)  
지출_총금액  
당월_매출_금액(분기당 매출금액을 싹 더하면 나올듯)  
(점포수 대비 매출이 중요)  

———  

6. (상권변화지표에서 따올칼럼들)   
운영_영업_개월_평균  
폐업_영업_개월_평균  
서울_운영_영업_개월_평균  
서울_폐업_영업_개월_평균  

*상권변화지표??-> 이거 필요한가? -> 필요할듯??*

상권_변화_지표  
상권_변화_지표_명    
**-> 결론적으로 폐업률을 구해야 함 ( 상권-점포 부분에 폐업률 칼럼이 있으나, 존나 대충나와서 걍 구하는게 빠를듯 )**  

`+@ : 상권 배후지랑 상권이랑 상관안하고 묶어서 보겠습니다!`

In [78]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc

In [79]:
# 상권-생활인구 , 상권배후지-생활인구
living_pop_1 =  pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권-생활인구).csv",encoding="EUC-KR")
living_pop_2 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권배후지-생활인구).csv",encoding="EUC-KR")
# 상권-집객시설, 상권배후지-집객시설
jipgaek_1 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권-집객시설).csv",encoding="EUC-KR")
jipgaek_2 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권배후지-집객시설).csv",encoding="EUC-KR")
# 상권-점포 , 상권배후지-점포 
jumpo_1 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권-점포).csv",encoding="EUC-KR")
jumpo_2 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권배후지-점포).csv",encoding="EUC-KR")
# 상권-아파트, 상권배후지-아파트 
apt_1 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권-아파트).csv",encoding="EUC-KR")
apt_2 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권배후지-아파트).csv",encoding="EUC-KR")
# 상권-추정매출, 상권배후지-추정매출 
est_sale_1 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권-추정매출).csv",encoding="EUC-KR")
est_sale_2 = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권배후지-추정매출).csv",encoding="EUC-KR")
# 상권-상권변화지표 
change_factor = pd.read_csv("data/서울시 우리마을가게 상권분석서비스(상권-상권변화지표).csv",encoding="EUC-KR")

In [80]:
print(len(living_pop_1.columns)) # 매우 많은 col갯수
print(len(living_pop_2.columns))
print(living_pop_1.count(), "\n")
print(living_pop_2.count())

532
532
기준 년코드                         40041
기준_분기_코드                       40041
상권_구분_코드                       40041
 상권_구분_코드_명                    40041
상권_코드                          40041
                               ...  
여성연령대_60_이상_일요일시간대_2_생활인구_수    40041
여성연령대_60_이상_일요일시간대_3_생활인구_수    40041
여성연령대_60_이상_일요일시간대_4_생활인구_수    40041
여성연령대_60_이상_일요일시간대_5_생활인구_수    40041
여성연령대_60_이상_일요일시간대_6_생활인구_수    40041
Length: 532, dtype: int64 

기준_년_코드                        21800
기준_분기_코드                       21800
상권_구분_코드                       21800
상권_구분_코드_명                     21800
상권_코드                          21800
                               ...  
여성연령대_60_이상_일요일시간대_2_생활인구_수    21800
여성연령대_60_이상_일요일시간대_3_생활인구_수    21800
여성연령대_60_이상_일요일시간대_4_생활인구_수    21800
여성연령대_60_이상_일요일시간대_5_생활인구_수    21800
여성연령대_60_이상_일요일시간대_6_생활인구_수    21800
Length: 532, dtype: int64


In [5]:
#print(jipgaek_1.count(), "\n")
#print(len(jipgaek_1.columns))
print(jipgaek_1.columns)
print(jipgaek_2.columns)
print(jipgaek_1.count())
print(jipgaek_2.count())

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '집객시설_수', '관공서_수', '은행_수', '종합병원_수', '일반_병원_수', '약국_수', '유치원_수',
       '초등학교_수', '중학교_수', '고등학교_수', '대학교_수', '백화점_수', '슈퍼마켓_수', '극장_수',
       '숙박_시설_수', '공항_수', '철도_역_수', '버스_터미널_수', '지하철_역_수', '버스_정거장_수'],
      dtype='object')
Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '집객시설_수', '관공서_수', '은행_수', '종합병원_수', '일반_병원_수', '약국_수', '유치원_수',
       '초등학교_수', '중학교_수', '고등학교_수', '대학교_수', '백화점_수', '슈퍼마켓_수', '극장_수',
       '숙박_시설_수', '공항_수', '철도_역_수', '버스_터미널_수', '지하철_역_수', '버스_정거장_수'],
      dtype='object')
기준_년_코드       38022
기준_분기_코드      38022
상권_구분_코드      38022
상권_구분_코드_명    38022
상권_코드         38022
상권_코드_명       38022
집객시설_수        38022
관공서_수         14628
은행_수          12623
종합병원_수          611
일반_병원_수        3971
약국_수          24972
유치원_수          5021
초등학교_수          215
중학교_수            51
고등학교_수          148
대학교_수          1071
백화점_수           336
슈퍼마켓_수      

In [81]:
print(jumpo_1.columns)
print(jumpo_2.columns)
print(jumpo_1.count())
print(jumpo_2.count())

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '서비스_업종_코드', '서비스_업종_코드_명', '점포_수', '유사_업종_점포_수', '개업_율', '개업_점포_수',
       '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수'],
      dtype='object')
Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '서비스_업종_코드', '서비스_업종_코드_명', '점포_수', '유사_업종_점포_수', '개업_율', '개업_점포_수',
       '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수'],
      dtype='object')
기준_년_코드        898068
기준_분기_코드       898068
상권_구분_코드       898068
상권_구분_코드_명     898068
상권_코드          898068
상권_코드_명        898068
서비스_업종_코드      898068
서비스_업종_코드_명    898068
점포_수           898068
유사_업종_점포_수     898068
개업_율           898068
개업_점포_수        898068
폐업_률           898068
폐업_점포_수        898068
프랜차이즈_점포_수     898068
dtype: int64
기준_년_코드        1037007
기준_분기_코드       1037007
상권_구분_코드       1037007
상권_구분_코드_명     1037007
상권_코드          1037007
상권_코드_명        1037007
서비스_업종_코드      1037007
서비스_업종_코드_명    1037007
점포_수           1037007
유사_업종_점포_수     10370

In [82]:
print(apt_1.columns)
print(apt_2.columns)
print(apt_1.count())
print(apt_2.count())

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '아파트_단지_수', '아파트_면적_66_제곱미터_미만_세대_수', '아파트_면적_66_제곱미터_세대_수',
       '아파트_면적_99_제곱미터_세대_수', '아파트_면적_132_제곱미터_세대_수', '아파트_면적_165_제곱미터_세대_수',
       '아파트_가격_1_억_미만_세대_수', '아파트_가격_1_억_세대_수', '아파트_가격_2_억_세대_수',
       '아파트_가격_3_억_세대_수', '아파트_가격_4_억_세대_수', '아파트_가격_5_억_세대_수',
       '아파트_가격_6_억_이상_세대_수', '아파트_평균_면적', '아파트_평균_시가'],
      dtype='object')
Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '아파트_단지_수', '아파트_면적_66_제곱미터_미만_세대_수', '아파트_면적_66_제곱미터_세대_수',
       '아파트_면적_99_제곱미터_세대_수', '아파트_면적_132_제곱미터_세대_수', '아파트_면적_165_제곱미터_세대_수',
       '아파트_가격_1_억_미만_세대_수', '아파트_가격_1_억_세대_수', '아파트_가격_2_억_세대_수',
       '아파트_가격_3_억_세대_수', '아파트_가격_4_억_세대_수', '아파트_가격_5_억_세대_수',
       '아파트_가격_6_억_이상_세대_수', '아파트_평균_면적', '아파트_평균_시가'],
      dtype='object')
기준_년_코드                   35121
기준_분기_코드                  35121
상권_구분_코드                  35121
상권_구분_코드_명                35121
상권_코드     

In [8]:
print(est_sale_1.columns)
print(est_sale_2.columns)
print(est_sale_1.count())
print(est_sale_2.count())

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '서비스_업종_코드', '서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수', '주중_매출_비율',
       '주말_매출_비율', '월요일_매출_비율', '화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
       '금요일_매출_비율', '토요일_매출_비율', '일요일_매출_비율', '시간대_00~06_매출_비율',
       '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
       '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율',
       '연령대_10_매출_비율', '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율',
       '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
       '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',

In [83]:
print(change_factor.columns)
print(change_factor.count())

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '상권_변화_지표', '상권_변화_지표_명', '운영_영업_개월_평균', '폐업_영업_개월_평균',
       '서울_운영_영업_개월_평균', '서울_폐업_영업_개월_평균'],
      dtype='object')
기준_년_코드           40020
기준_분기_코드          40020
상권_구분_코드          40020
상권_구분_코드_명        40020
상권_코드             40020
상권_코드_명           40020
상권_변화_지표          40020
상권_변화_지표_명        40020
운영_영업_개월_평균       40020
폐업_영업_개월_평균       40020
서울_운영_영업_개월_평균    40020
서울_폐업_영업_개월_평균    40020
dtype: int64


In [84]:
#living_pop_1
#living_pop_2
print("상권 - 생활인구's shape : ", living_pop_1.shape)
print("상권배후지 - 생활인구's shape : ", living_pop_2.shape)
#jipgaek_1
#jipgaek_2
print("상권 - 집객시설's shape : ", jipgaek_1.shape)
print("상권배후지 - 집객시설's shape : ", jipgaek_2.shape)

#jumpo_1
#jumpo_2
print("상권 - 점포's shape : ", jumpo_1.shape)
print("상권배후지 - 점포's shape : ", jumpo_2.shape)


#apt_1
#apt_2
print("상권 - 아파트's shape : ", apt_1.shape)
print("상권배후지 - 아파트's shape : ", apt_2.shape)

#est_sale_1
#est_sale_2
print("상권 - 추정매출's shape : ", est_sale_1.shape)
print("상권배후지 - 추정매출's shape : ", est_sale_2.shape)

#change_factor
print("상권 - 상권변화지표's shape : ", change_factor.shape)

상권 - 생활인구's shape :  (40041, 532)
상권배후지 - 생활인구's shape :  (21800, 532)
상권 - 집객시설's shape :  (38022, 26)
상권배후지 - 집객시설's shape :  (26160, 26)
상권 - 점포's shape :  (898068, 15)
상권배후지 - 점포's shape :  (1037007, 15)
상권 - 아파트's shape :  (35121, 21)
상권배후지 - 아파트's shape :  (26112, 21)
상권 - 추정매출's shape :  (136820, 80)
상권배후지 - 추정매출's shape :  (189413, 80)
상권 - 상권변화지표's shape :  (40020, 12)


In [85]:
print(living_pop_2.columns[0])
print(living_pop_1.columns[0])

print(living_pop_2.columns[3])
print(living_pop_1.columns[3])


기준_년_코드
기준 년코드
상권_구분_코드_명
 상권_구분_코드_명


In [86]:

living_pop_1.rename(columns = {"기준 년코드": "기준_년_코드"}, inplace = True)

In [87]:
living_pop_1.rename(columns = {" 상권_구분_코드_명": "상권_구분_코드_명"},inplace = True)

In [88]:
print(living_pop_1.columns[0])
print(living_pop_2.columns[0])
print(living_pop_1.columns[0] == living_pop_2.columns[0])
print(living_pop_2.columns[3])
print(living_pop_1.columns[3])
print(living_pop_1.columns[3] == living_pop_2.columns[3])


기준_년_코드
기준_년_코드
True
상권_구분_코드_명
상권_구분_코드_명
True


In [89]:
# 배후지 + 상권 concat (생활인구)
living_pop = pd.concat([living_pop_1,living_pop_2])
living_pop

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,...,여성연령대_60_이상_토요일시간대_3_생활인구_수,여성연령대_60_이상_토요일시간대_4_생활인구_수,여성연령대_60_이상_토요일시간대_5_생활인구_수,여성연령대_60_이상_토요일시간대_6_생활인구_수,여성연령대_60_이상_일요일시간대_1_생활인구_수,여성연령대_60_이상_일요일시간대_2_생활인구_수,여성연령대_60_이상_일요일시간대_3_생활인구_수,여성연령대_60_이상_일요일시간대_4_생활인구_수,여성연령대_60_이상_일요일시간대_5_생활인구_수,여성연령대_60_이상_일요일시간대_6_생활인구_수
0,2022,4,U,관광특구,1001496,강남 마이스 관광특구,112658,55901,56757,8418,...,185,181,158,46,66,115,162,158,132,39
1,2022,4,U,관광특구,1001495,잠실 관광특구,3632727,1757144,1875581,420460,...,6788,6949,8331,4315,7017,6913,6385,6565,7523,4113
2,2022,4,U,관광특구,1001494,종로?청계 관광특구,3957665,2127522,1830142,153739,...,13096,12960,9609,3662,4754,6670,8433,8798,7059,3777
3,2022,4,U,관광특구,1001493,동대문패션타운 관광특구,3249964,1534697,1715268,159801,...,7121,6495,5297,3077,5431,5784,5412,5078,6039,5319
4,2022,4,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,4667755,2345370,2322385,228777,...,15336,15293,8499,2011,2675,6137,8875,7776,5336,1695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21795,2018,1,A,골목상권,2110005,세검정,1059948,464219,595728,166533,...,3256,3197,4479,3524,6586,5684,3575,3411,4758,3543
21796,2018,1,A,골목상권,2110004,대신고등학교,3236420,1458423,1777999,627940,...,9361,8911,11898,9244,19328,16303,9800,8936,12163,9549
21797,2018,1,A,골목상권,2110003,세검정초등학교,1276088,566551,709536,207655,...,3867,3739,5164,4295,8646,7215,4358,4034,5589,4389
21798,2018,1,A,골목상권,2110002,독립문역 1번,1733119,753770,979349,292600,...,5716,5449,7214,6027,12022,10305,5546,5370,7211,6055


In [90]:
jipgaek_1.columns == jipgaek_2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [91]:
# 배후지 + 상권 concat (집객시설)
jipgaek = pd.concat([jipgaek_1,jipgaek_2])
jipgaek

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,...,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
0,2022,4,A,골목상권,2110332,돌곶이역 3번,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,2022,4,R,전통시장,2130133,우이골목시장(우이시장),4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,4,A,골목상권,2110367,강북구청,13,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,2022,4,A,골목상권,2110797,KT대방빌딩,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,2022,4,R,전통시장,2130277,조원동 펭귄시장(신림중앙시장),2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26155,2017,1,A,골목상권,2110435,서울과학기술대,47,1.0,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,29.0
26156,2017,1,A,골목상권,2110282,국민대학교앞,28,3.0,1.0,NaN,...,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,15.0
26157,2017,1,A,골목상권,2110233,중랑역 1번,71,1.0,1.0,NaN,...,NaN,NaN,1.0,1.0,9.0,NaN,NaN,NaN,1.0,14.0
26158,2017,1,A,골목상권,2110214,경희대후문,68,2.0,3.0,NaN,...,4.0,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,21.0


In [92]:
jumpo_1.columns == jumpo_2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [93]:
# 배후지 + 상권 concat (점포)
jumpo= pd.concat([jumpo_1,jumpo_2])
jumpo

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
0,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,6,6,0,0,0,0,0
1,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300042,주유소,1,1,0,0,0,0,0
2,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300041,예술품,10,10,0,0,0,0,0
3,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300038,자동차부품,10,10,0,0,0,0,0
4,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300036,조명용품,14,14,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037002,2020,1,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,6,6,0,0,17,1,0
1037003,2020,1,A,골목상권,2110001,이북5도청사,CS100005,제과점,0,1,0,0,0,0,1
1037004,2020,1,A,골목상권,2110001,이북5도청사,CS100004,양식음식점,2,2,0,0,0,0,0
1037005,2020,1,A,골목상권,2110001,이북5도청사,CS100003,일식음식점,1,1,0,0,0,0,0


In [94]:
apt_1.columns == apt_2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [95]:
# 배후지 + 상권 concat (아파트)
apt= pd.concat([apt_1,apt_2])
apt

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,아파트_단지_수,아파트_면적_66_제곱미터_미만_세대_수,아파트_면적_66_제곱미터_세대_수,아파트_면적_99_제곱미터_세대_수,...,아파트_면적_165_제곱미터_세대_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,아파트_평균_면적,아파트_평균_시가
0,2022,4,R,전통시장,2130326,고덕 골목형상점가,2,657.0,5.0,NaN,...,NaN,NaN,2.0,4.0,NaN,129.0,21.0,506.0,53,633301800
1,2022,3,R,전통시장,2130326,고덕 골목형상점가,2,657.0,5.0,NaN,...,NaN,NaN,2.0,4.0,NaN,129.0,21.0,506.0,53,633301800
2,2022,2,R,전통시장,2130326,고덕 골목형상점가,2,657.0,5.0,NaN,...,NaN,NaN,2.0,4.0,NaN,129.0,21.0,506.0,53,633301800
3,2022,1,R,전통시장,2130326,고덕 골목형상점가,2,657.0,5.0,NaN,...,NaN,NaN,2.0,4.0,NaN,129.0,21.0,506.0,53,633301800
4,2022,4,R,전통시장,2130325,명일전통시장,5,203.0,372.0,NaN,...,NaN,3.0,20.0,1.0,NaN,123.0,172.0,256.0,61,294384895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26107,2017,1,A,골목상권,2111084,길동신동아아파트,133,1956.0,1626.0,207.0,...,18.0,513.0,877.0,1826.0,549.0,126.0,1.0,0.0,58,161129693
26108,2017,1,A,골목상권,2111085,묘곡초등학교(고덕아이파크아파트),53,785.0,603.0,209.0,...,78.0,3.0,288.0,93.0,270.0,448.0,377.0,317.0,64,266704264
26109,2017,1,A,골목상권,2111086,한영중고,23,91.0,1873.0,436.0,...,28.0,9.0,82.0,7.0,1394.0,842.0,476.0,60.0,92,388268946
26110,2017,1,A,골목상권,2111087,고덕중학교(고덕2동주민센터),29,189.0,1.0,0.0,...,0.0,5.0,182.0,3.0,0.0,0.0,0.0,0.0,44,143163109


In [96]:
est_sale_1.columns == est_sale_2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [97]:
# 배후지 + 상권 concat (추정매출)
est_sale= pd.concat([est_sale_1,est_sale_2])
est_sale

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,4770649,1016,...,0,34,982,77,779,87,24,44,5,6
1,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,1364810577,4605,...,132,2208,1920,90,1044,1278,894,666,156,8
2,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,8512320761,3912,...,0,1421,2491,0,106,212,781,785,2028,6
3,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,658389405,9080,...,36,4594,3400,54,2085,2412,1687,571,1183,5
4,2022,4,U,관광특구,1001496,강남 마이스 관광특구,CS300026,완구,383996294,3093,...,0,1687,1349,228,857,687,1063,200,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189408,2022,1,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,45103003,5237,...,0,2466,2534,64,621,1087,889,1586,754,9
189409,2022,1,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,110443640,2916,...,1045,2231,619,52,262,634,584,730,588,10
189410,2022,1,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,134725445,13966,...,0,7768,5254,264,686,1534,3482,4137,2920,8
189411,2022,1,A,골목상권,2110001,이북5도청사,CS100005,제과점,135143932,14609,...,27,7057,7552,105,1846,2391,4346,3622,2299,*


In [99]:
change_factor

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2022,4,U,관광특구,1001496,강남 마이스 관광특구,HH,정체,146,65,123,56
1,2022,4,U,관광특구,1001495,잠실 관광특구,LL,다이나믹,118,55,123,56
2,2022,4,U,관광특구,1001494,종로?청계 관광특구,HH,정체,165,75,123,56
3,2022,4,U,관광특구,1001493,동대문패션타운 관광특구,HH,정체,151,65,123,56
4,2022,4,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,HH,정체,170,66,123,56
...,...,...,...,...,...,...,...,...,...,...,...,...
40015,2017,1,A,골목상권,2110005,세검정,HH,정체,105,56,98,51
40016,2017,1,A,골목상권,2110004,대신고등학교,HH,정체,149,66,98,51
40017,2017,1,A,골목상권,2110003,세검정초등학교,HH,정체,104,55,98,51
40018,2017,1,A,골목상권,2110002,독립문역 1번,HH,정체,116,55,98,51


In [100]:
#living_pop
print("생활인구's shape : ", living_pop.shape)

#jipgaek
print("집객시설's shape : ", jipgaek_1.shape)

#jumpo
print("점포's shape : ", jumpo_1.shape)

#apt
print("아파트's shape : ", apt_1.shape)

#est_sale
print("추정매출's shape : ", est_sale_1.shape)

#change_factor
print("상권변화지표's shape : ", change_factor.shape) # 얘는 똑같음

생활인구's shape :  (61841, 532)
집객시설's shape :  (38022, 26)
점포's shape :  (898068, 15)
아파트's shape :  (35121, 21)
추정매출's shape :  (136820, 80)
상권변화지표's shape :  (40020, 12)


In [101]:
# living_pop에서 가져올 columns
# 기준_년_코드 , 상권_구분_코드 ,상권_구분_코드_명 , 상권_코드 , 상권_코드_명 ,생활_인구_수
# 다른 내용들을 더 가져와서 비교해야 할 경우, 그때 수정하겠습니다.
living_pop = living_pop[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드','상권_코드_명','총_생활인구_수']]
living_pop

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수
0,2022,U,관광특구,1001496,강남 마이스 관광특구,112658
1,2022,U,관광특구,1001495,잠실 관광특구,3632727
2,2022,U,관광특구,1001494,종로?청계 관광특구,3957665
3,2022,U,관광특구,1001493,동대문패션타운 관광특구,3249964
4,2022,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,4667755
...,...,...,...,...,...,...
21795,2018,A,골목상권,2110005,세검정,1059948
21796,2018,A,골목상권,2110004,대신고등학교,3236420
21797,2018,A,골목상권,2110003,세검정초등학교,1276088
21798,2018,A,골목상권,2110002,독립문역 1번,1733119


In [102]:
# jipgaek에서 가져올 columns
# 기준_년_코드 , 상권_구분_코드, 상권_구분_코드_명 , 상권_코드 ,상권_코드_명, 집객시설_수
# 다른 내용들을 더 가져와서 비교해야 할 경우, 그때 수정하겠습니다.
jipgaek = jipgaek[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드','상권_코드_명','집객시설_수']]
jipgaek

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수
0,2022,A,골목상권,2110332,돌곶이역 3번,2
1,2022,R,전통시장,2130133,우이골목시장(우이시장),4
2,2022,A,골목상권,2110367,강북구청,13
3,2022,A,골목상권,2110797,KT대방빌딩,8
4,2022,R,전통시장,2130277,조원동 펭귄시장(신림중앙시장),2
...,...,...,...,...,...,...
26155,2017,A,골목상권,2110435,서울과학기술대,47
26156,2017,A,골목상권,2110282,국민대학교앞,28
26157,2017,A,골목상권,2110233,중랑역 1번,71
26158,2017,A,골목상권,2110214,경희대후문,68


In [103]:
# jumpo에서 가져올 columns
# 기준_년_코드 , 상권_구분_코드, 상권_구분_코드_명 , 상권_코드 ,상권_코드_명, 서비스_업종_코드, 서비스_업종_코드_명, 점포수
# 다른 내용들을 더 가져와서 비교해야 할 경우, 그때 수정하겠습니다.
jumpo = jumpo[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드','상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','점포_수']]
jumpo

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수
0,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,6
1,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300042,주유소,1
2,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300041,예술품,10
3,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300038,자동차부품,10
4,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300036,조명용품,14
...,...,...,...,...,...,...,...,...
1037002,2020,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,6
1037003,2020,A,골목상권,2110001,이북5도청사,CS100005,제과점,0
1037004,2020,A,골목상권,2110001,이북5도청사,CS100004,양식음식점,2
1037005,2020,A,골목상권,2110001,이북5도청사,CS100003,일식음식점,1


In [104]:
# apt에서 가져올 columns
# 기준_년_코드 , 상권_구분_코드, 상권_구분_코드_명 , 상권_코드 ,상권_코드_명, 아파트_평균_면적, 아파트_평균_시가
# 다른 내용들을 더 가져와서 비교해야 할 경우, 그때 수정하겠습니다.
apt = apt[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드','상권_코드_명','아파트_평균_면적','아파트_평균_시가']]
apt

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,아파트_평균_면적,아파트_평균_시가
0,2022,R,전통시장,2130326,고덕 골목형상점가,53,633301800
1,2022,R,전통시장,2130326,고덕 골목형상점가,53,633301800
2,2022,R,전통시장,2130326,고덕 골목형상점가,53,633301800
3,2022,R,전통시장,2130326,고덕 골목형상점가,53,633301800
4,2022,R,전통시장,2130325,명일전통시장,61,294384895
...,...,...,...,...,...,...,...
26107,2017,A,골목상권,2111084,길동신동아아파트,58,161129693
26108,2017,A,골목상권,2111085,묘곡초등학교(고덕아이파크아파트),64,266704264
26109,2017,A,골목상권,2111086,한영중고,92,388268946
26110,2017,A,골목상권,2111087,고덕중학교(고덕2동주민센터),44,143163109


In [105]:
# est_sale에서 가져올 columns
# 기준_년_코드 , 상권_구분_코드, 상권_구분_코드_명 , 상권_코드 ,상권_코드_명, 서비스_업종_코드, 서비스_업종_코드_명,분기당_매출_금액, 분기당_매출_건수
# 다른 내용들을 더 가져와서 비교해야 할 경우, 그때 수정하겠습니다.
est_sale= est_sale[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드','상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','분기당_매출_금액','분기당_매출_건수' ]]
est_sale

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수
0,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,4770649,1016
1,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,1364810577,4605
2,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,8512320761,3912
3,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,658389405,9080
4,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300026,완구,383996294,3093
...,...,...,...,...,...,...,...,...,...
189408,2022,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,45103003,5237
189409,2022,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,110443640,2916
189410,2022,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,134725445,13966
189411,2022,A,골목상권,2110001,이북5도청사,CS100005,제과점,135143932,14609


In [31]:
# est_sale은 2022년 데이터밖에 없으므로 이전년도 데이터들도 다운받아서 똑같은 작업 수행

In [106]:
# change_factor에서 가져올 columns
# 얘는 다 필요할 col같아서 그대로 두었습니다.
# 필요한 column들 : 기준_년_코드 , 상권_구분_코드, 상권_구분_코드_명 , 상권_코드 ,상권_코드_명, 상권_변화_지표,상권_변화_지표_명
# 운영_영업_개월_평균, 폐업_영업_개월_평균 , 서울_운영_영업_개월_평균, 서울_폐업_영업_개월_평균
change_factor

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2022,4,U,관광특구,1001496,강남 마이스 관광특구,HH,정체,146,65,123,56
1,2022,4,U,관광특구,1001495,잠실 관광특구,LL,다이나믹,118,55,123,56
2,2022,4,U,관광특구,1001494,종로?청계 관광특구,HH,정체,165,75,123,56
3,2022,4,U,관광특구,1001493,동대문패션타운 관광특구,HH,정체,151,65,123,56
4,2022,4,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,HH,정체,170,66,123,56
...,...,...,...,...,...,...,...,...,...,...,...,...
40015,2017,1,A,골목상권,2110005,세검정,HH,정체,105,56,98,51
40016,2017,1,A,골목상권,2110004,대신고등학교,HH,정체,149,66,98,51
40017,2017,1,A,골목상권,2110003,세검정초등학교,HH,정체,104,55,98,51
40018,2017,1,A,골목상권,2110002,독립문역 1번,HH,정체,116,55,98,51


In [107]:
living_pop_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(구_상권_생활인구)_2017년.csv",encoding="EUC-KR")
living_pop_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(구_상권배후지_생활인구)_2017년.csv",encoding="EUC-KR")

#기준_년_코드 , 상권_구분_코드 ,상권_구분_코드_명 , 상권_코드 , 상권_코드_명 ,생활_인구_수

In [108]:
living_pop_temp_1['기준 년코드']
living_pop_temp_2['기준_년_코드']

living_pop_temp_1[' 상권_구분_코드_명']
living_pop_temp_2['상권_구분_코드_명']

living_pop_temp_1['상권_구분_코드']
living_pop_temp_2['상권_구분_코드']

living_pop_temp_1['상권_코드']
living_pop_temp_2['상권_코드']

living_pop_temp_1['상권_코드.1']
living_pop_temp_2['상권_코드_명']

living_pop_temp_1['남성_생활인구_수'] # 남성+여성 해야할듯?
living_pop_temp_1['여성_생활인구_수']

living_pop_temp_2['남성_생활인구_수'] # 얘도 
living_pop_temp_2['여성_생활인구_수']

# 이렇게 뽑아내서 잘 조무리면 될듯




0        217723
1        829099
2        509224
3        450726
4          8164
         ...   
4035     682653
4036      52632
4037    1739889
4038    1437373
4039    1218058
Name: 여성_생활인구_수, Length: 4040, dtype: int64

In [109]:
living_pop_temp_1 = living_pop_temp_1[['기준 년코드',' 상권_구분_코드_명','상권_구분_코드','상권_코드','상권_코드.1','남성_생활인구_수','여성_생활인구_수']]
living_pop_temp_2 = living_pop_temp_2[['기준_년_코드','상권_구분_코드_명','상권_구분_코드','상권_코드','상권_코드_명','남성_생활인구_수','여성_생활인구_수']]

In [110]:
living_pop_temp_1

,기준 년코드,상권_구분_코드_명,상권_구분_코드,상권_코드,상권_코드.1,남성_생활인구_수,여성_생활인구_수
0,2017,골목상권,A,1000001,계동길,76503,18919
1,2017,골목상권,A,1000002,난계로27길,84020,9451
2,2017,골목상권,A,1000003,돈화문로11가길,35428,977
3,2017,골목상권,A,1000004,명륜길,108024,32635
4,2017,골목상권,A,1000005,백석동길,1784,1
...,...,...,...,...,...,...,...
5975,2017,관광특구,U,1001492,명동 남대문 북창동 다동 무교동 관광특구,3585270,176120
5976,2017,관광특구,U,1001493,동대문패션타운 관광특구,1457071,98390
5977,2017,관광특구,U,1001494,종로·청계 관광특구,2542565,153520
5978,2017,관광특구,U,1001495,잠실 관광특구,897799,69435


In [111]:
living_pop_temp_2

,기준_년_코드,상권_구분_코드_명,상권_구분_코드,상권_코드,상권_코드_명,남성_생활인구_수,여성_생활인구_수
0,2017,골목상권,A,1000001,계동길,284549,217723
1,2017,골목상권,A,1000002,난계로27길,1020808,829099
2,2017,골목상권,A,1000003,돈화문로11가길,794470,509224
3,2017,골목상권,A,1000004,명륜길,547947,450726
4,2017,골목상권,A,1000005,백석동길,4839,8164
...,...,...,...,...,...,...,...
4035,2017,골목상권,A,1001006,천호대로197길,684478,682653
4036,2017,골목상권,A,1001007,천호대로219길,77063,52632
4037,2017,골목상권,A,1001008,천호옛14길,1597073,1739889
4038,2017,골목상권,A,1001009,천호옛길,1423582,1437373


In [114]:
living_pop_temp_1['총_생활인구_수'] = living_pop_temp_1['남성_생활인구_수']+living_pop_temp_1['여성_생활인구_수']
living_pop_temp_1 = living_pop_temp_1.drop(["남성_생활인구_수","여성_생활인구_수"],axis=1)

In [116]:
living_pop_temp_2['총_생활인구_수'] = living_pop_temp_2['남성_생활인구_수']+living_pop_temp_2['여성_생활인구_수']
living_pop_temp_2 = living_pop_temp_2.drop(["남성_생활인구_수","여성_생활인구_수"],axis=1)

In [120]:
living_pop_temp_1 = living_pop_temp_1[['기준 년코드','상권_구분_코드',' 상권_구분_코드_명','상권_코드','상권_코드.1','총_생활인구_수']]
living_pop_temp_1

,기준 년코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드.1,총_생활인구_수
0,2017,A,골목상권,1000001,계동길,95422
1,2017,A,골목상권,1000002,난계로27길,93471
2,2017,A,골목상권,1000003,돈화문로11가길,36405
3,2017,A,골목상권,1000004,명륜길,140659
4,2017,A,골목상권,1000005,백석동길,1785
...,...,...,...,...,...,...
5975,2017,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,3761390
5976,2017,U,관광특구,1001493,동대문패션타운 관광특구,1555461
5977,2017,U,관광특구,1001494,종로·청계 관광특구,2696085
5978,2017,U,관광특구,1001495,잠실 관광특구,967234


In [123]:
living_pop_temp_2 = living_pop_temp_2[['기준_년_코드','상권_구분_코드','상권_구분_코드_명','상권_코드','상권_코드_명','총_생활인구_수']]
living_pop_temp_2

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수
0,2017,A,골목상권,1000001,계동길,502272
1,2017,A,골목상권,1000002,난계로27길,1849907
2,2017,A,골목상권,1000003,돈화문로11가길,1303694
3,2017,A,골목상권,1000004,명륜길,998673
4,2017,A,골목상권,1000005,백석동길,13003
...,...,...,...,...,...,...
4035,2017,A,골목상권,1001006,천호대로197길,1367131
4036,2017,A,골목상권,1001007,천호대로219길,129695
4037,2017,A,골목상권,1001008,천호옛14길,3336962
4038,2017,A,골목상권,1001009,천호옛길,2860955


In [118]:
living_pop

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수
0,2022,U,관광특구,1001496,강남 마이스 관광특구,112658
1,2022,U,관광특구,1001495,잠실 관광특구,3632727
2,2022,U,관광특구,1001494,종로?청계 관광특구,3957665
3,2022,U,관광특구,1001493,동대문패션타운 관광특구,3249964
4,2022,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,4667755
...,...,...,...,...,...,...
21795,2018,A,골목상권,2110005,세검정,1059948
21796,2018,A,골목상권,2110004,대신고등학교,3236420
21797,2018,A,골목상권,2110003,세검정초등학교,1276088
21798,2018,A,골목상권,2110002,독립문역 1번,1733119


In [124]:
living_pop_temp_1.rename(columns = {"기준 년코드": "기준_년_코드"}, inplace = True)
living_pop_temp_1.rename(columns = {" 상권_구분_코드_명": "상권_구분_코드_명"}, inplace = True)
living_pop_temp_1.rename(columns = {"상권_코드.1": "상권_코드_명"}, inplace = True)

In [125]:
living_pop_temp = pd.concat([living_pop_temp_1 , living_pop_temp_2])

In [126]:
living_pop = pd.concat([living_pop_temp , living_pop])

In [127]:
living_pop

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수
0,2017,A,골목상권,1000001,계동길,95422
1,2017,A,골목상권,1000002,난계로27길,93471
2,2017,A,골목상권,1000003,돈화문로11가길,36405
3,2017,A,골목상권,1000004,명륜길,140659
4,2017,A,골목상권,1000005,백석동길,1785
...,...,...,...,...,...,...
21795,2018,A,골목상권,2110005,세검정,1059948
21796,2018,A,골목상권,2110004,대신고등학교,3236420
21797,2018,A,골목상권,2110003,세검정초등학교,1276088
21798,2018,A,골목상권,2110002,독립문역 1번,1733119


In [130]:
living_pop[living_pop['기준_년_코드']==2019] # 잘 나오는군요

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수
20020,2019,U,관광특구,1001496,강남 마이스 관광특구,130493
20021,2019,U,관광특구,1001495,잠실 관광특구,3225556
20022,2019,U,관광특구,1001494,종로?청계 관광특구,4917960
20023,2019,U,관광특구,1001493,동대문패션타운 관광특구,3875197
20024,2019,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,5278508
...,...,...,...,...,...,...
17435,2019,A,골목상권,2110005,세검정,836304
17436,2019,A,골목상권,2110004,대신고등학교,3070855
17437,2019,A,골목상권,2110003,세검정초등학교,1235784
17438,2019,A,골목상권,2110002,독립문역 1번,1691301


In [131]:
jumpo_2017_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권_점포)_2017년.csv",encoding="EUC-KR")
jumpo_2018_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권_점포)_2018년.csv",encoding="EUC-KR")
jumpo_2019_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권_점포)_2019년.csv",encoding="EUC-KR")


jumpo_2017_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권배후지_점포)_2017년.csv",encoding="EUC-KR")
jumpo_2018_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권배후지_점포)_2018년.csv",encoding="EUC-KR")
jumpo_2019_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권배후지_점포)_2019년.csv",encoding="EUC-KR")



In [132]:
jumpo_2017_temp_1=jumpo_2017_temp_1[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드' , '상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','점포_수']]
jumpo_2017_temp_2=jumpo_2017_temp_2[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드' , '상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','점포_수']]
jumpo_2018_temp_1=jumpo_2018_temp_1[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드' , '상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','점포_수']]
jumpo_2018_temp_2=jumpo_2018_temp_2[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드' , '상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','점포_수']]
jumpo_2019_temp_1=jumpo_2019_temp_1[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드' , '상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','점포_수']]
jumpo_2019_temp_2=jumpo_2017_temp_2[['기준_년_코드' , '상권_구분_코드' , '상권_구분_코드_명' , '상권_코드' , '상권_코드_명','서비스_업종_코드','서비스_업종_코드_명','점포_수']]
jumpo_2017 = pd.concat([jumpo_2017_temp_1 , jumpo_2017_temp_2])
jumpo_2018 = pd.concat([jumpo_2018_temp_1 , jumpo_2018_temp_2])
jumpo_2019 = pd.concat([jumpo_2019_temp_1 , jumpo_2019_temp_2])
jumpo_list = [jumpo_2017 , jumpo_2018,jumpo_2019, jumpo]
jumpo= pd.concat(jumpo_list ,ignore_index=True)

In [133]:
jumpo

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수
0,2017,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,12
1,2017,A,골목상권,2110001,이북5도청사,CS100007,치킨전문점,1
2,2017,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,3
3,2017,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,3
4,2017,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,3
...,...,...,...,...,...,...,...,...
3862295,2020,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,6
3862296,2020,A,골목상권,2110001,이북5도청사,CS100005,제과점,0
3862297,2020,A,골목상권,2110001,이북5도청사,CS100004,양식음식점,2
3862298,2020,A,골목상권,2110001,이북5도청사,CS100003,일식음식점,1


In [134]:
# 이제 추정매출 싹 정리해야합니다. 
# 현재 est_sale은 2022년만 있습니다. 2017~2021년도 데이터도 똑같이 해주면 됩니다.
est_sale

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수
0,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,4770649,1016
1,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,1364810577,4605
2,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,8512320761,3912
3,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,658389405,9080
4,2022,U,관광특구,1001496,강남 마이스 관광특구,CS300026,완구,383996294,3093
...,...,...,...,...,...,...,...,...,...
189408,2022,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,45103003,5237
189409,2022,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,110443640,2916
189410,2022,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,134725445,13966
189411,2022,A,골목상권,2110001,이북5도청사,CS100005,제과점,135143932,14609


In [135]:
est_2017_sale_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2017년.csv",encoding="EUC-KR")
est_2017_sale_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권배후지_추정매출)_2017년.csv",encoding="EUC-KR")

est_2018_sale_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2018년.csv",encoding="EUC-KR")
est_2018_sale_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권배후지_추정매출)_2018년.csv",encoding="EUC-KR")

est_2019_sale_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2019년.csv",encoding="EUC-KR")
est_2019_sale_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권배후지_추정매출)_2019년.csv",encoding="EUC-KR")

est_2020_sale_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2020년.csv",encoding="EUC-KR")
est_2020_sale_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권배후지_추정매출)_2020년.csv",encoding="EUC-KR")

est_2021_sale_temp_1 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_2021년.csv",encoding="EUC-KR")
est_2021_sale_temp_2 = pd.read_csv("data/서울시_우리마을가게_상권분석서비스(신_상권배후지_추정매출)_2021년.csv",encoding="EUC-KR")


In [136]:
est_sale.columns

Index(['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드',
       '서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수'],
      dtype='object')

In [138]:
est_2017_sale_temp_1 = est_2017_sale_temp_1[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2017_sale_temp_2 = est_2017_sale_temp_2[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2018_sale_temp_1 = est_2018_sale_temp_1[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2018_sale_temp_2 = est_2018_sale_temp_2[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2019_sale_temp_1 = est_2019_sale_temp_1[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2019_sale_temp_2 = est_2019_sale_temp_2[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2020_sale_temp_1 = est_2020_sale_temp_1[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2020_sale_temp_2 = est_2020_sale_temp_2[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2021_sale_temp_1 = est_2021_sale_temp_1[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]
est_2021_sale_temp_2 = est_2021_sale_temp_2[['기준_년_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드','서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수']]

In [139]:
# 배후지 + 상권 concat (추정매출)
est_sale_list = [est_2017_sale_temp_1,est_2017_sale_temp_2,
                 est_2018_sale_temp_1,est_2018_sale_temp_2,
                 est_2019_sale_temp_1,est_2019_sale_temp_2,
                 est_2020_sale_temp_1,est_2020_sale_temp_2,
                 est_2021_sale_temp_1,est_2021_sale_temp_2,
                 est_sale
                ]

est_sale= pd.concat(est_sale_list,ignore_index=True)

In [140]:
est_sale

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수
0,2017,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,816890247,28082
1,2017,A,골목상권,2110001,이북5도청사,CS100007,치킨전문점,19223252,517
2,2017,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,75483418,5755
3,2017,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,46115226,1306
4,2017,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,22041409,1607
...,...,...,...,...,...,...,...,...,...
2178583,2022,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,45103003,5237
2178584,2022,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,110443640,2916
2178585,2022,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,134725445,13966
2178586,2022,A,골목상권,2110001,이북5도청사,CS100005,제과점,135143932,14609


#### 지금까지 한게 뭐냐면  
1. 년도 일치  
2. 각각의 df에서 필요할 것 같은 column들만 추출  


In [146]:
living_pop[living_pop['기준_년_코드']==2019]

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수
20020,2019,U,관광특구,1001496,강남 마이스 관광특구,130493
20021,2019,U,관광특구,1001495,잠실 관광특구,3225556
20022,2019,U,관광특구,1001494,종로?청계 관광특구,4917960
20023,2019,U,관광특구,1001493,동대문패션타운 관광특구,3875197
20024,2019,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,5278508
...,...,...,...,...,...,...
17435,2019,A,골목상권,2110005,세검정,836304
17436,2019,A,골목상권,2110004,대신고등학교,3070855
17437,2019,A,골목상권,2110003,세검정초등학교,1235784
17438,2019,A,골목상권,2110002,독립문역 1번,1691301


In [152]:
#df_s = df.sort_values(by=["c1"], ascending=[True]) 
living_pop = living_pop.sort_values(by=["기준_년_코드"], ascending=[True])
jipgaek = jipgaek.sort_values(by=["기준_년_코드"] , ascending=[True])
jumpo = jumpo.sort_values(by=["기준_년_코드"] , ascending=[True])
apt= apt.sort_values(by=["기준_년_코드"] , ascending=[True])
est_sale= est_sale.sort_values(by=["기준_년_코드"] , ascending=[True])
change_factor= change_factor.sort_values(by=["기준_년_코드"] , ascending=[True])

In [154]:
# 최종적인 dataframe들의 shape
#living_pop
print("생활인구's shape : ", living_pop.shape)

#jipgaek
print("집객시설's shape : ", jipgaek.shape)

#jumpo
print("점포's shape : ", jumpo.shape)

#apt
print("아파트's shape : ", apt.shape)

#est_sale
print("추정매출's shape : ", est_sale.shape)

#change_factor
print("상권변화지표's shape : ", change_factor.shape) 

생활인구's shape :  (71861, 6)
집객시설's shape :  (64182, 6)
점포's shape :  (3862300, 8)
아파트's shape :  (61233, 7)
추정매출's shape :  (2178588, 9)
상권변화지표's shape :  (40020, 12)


아직 전처리 1단계라서 각각의 csv 이름들 뒤에 '_1' 을 붙혀두었습니다.

In [169]:
living_pop.to_csv('/Users/junny/Downloads/쭌/쭌_학교관련/23_1_수업/캡디/5주차/KimJuneHyun_EDA/preprocessed_data/living_pop_1.csv',index=None)
jipgaek.to_csv('/Users/junny/Downloads/쭌/쭌_학교관련/23_1_수업/캡디/5주차/KimJuneHyun_EDA/preprocessed_data/jipgaek_1.csv',index=None)
jumpo.to_csv('/Users/junny/Downloads/쭌/쭌_학교관련/23_1_수업/캡디/5주차/KimJuneHyun_EDA/preprocessed_data/jumpo_1.csv',index=None)
apt.to_csv('/Users/junny/Downloads/쭌/쭌_학교관련/23_1_수업/캡디/5주차/KimJuneHyun_EDA/preprocessed_data/apt_1.csv',index=None)
est_sale.to_csv('/Users/junny/Downloads/쭌/쭌_학교관련/23_1_수업/캡디/5주차/KimJuneHyun_EDA/preprocessed_data/est_sale_1.csv',index=None)
change_factor.to_csv('/Users/junny/Downloads/쭌/쭌_학교관련/23_1_수업/캡디/5주차/KimJuneHyun_EDA/preprocessed_data/change_factor_1.csv',index=None)

나중에 read_csv할때, encoding = "UTF-8"로 하시면 됩니다.  
```python
pd.read_csv("preprocessed_data/hello_world.csv",encoding="UTF-8")
```
이런식으로요